<br>  

## <span style='color:blue'>Section 1: Import</span>  

In [1]:
import pandas as pd
import numpy as np
import copy

from sklearn.model_selection import StratifiedKFold

from sklearn.impute import KNNImputer
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import StandardScaler
from collections import Counter

from sklearn.model_selection import GridSearchCV
from imblearn.pipeline import Pipeline

from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score

<br>  

## <span style='color:blue'>Section 2: Read, drop columns, form X_train and y_train</span>  

In [2]:
# ---------- read ----------

df_1 = pd.read_csv('../data/code_3_train.csv')

# ---------- drop ----------

print('Before drop :', df_1.shape)
df_1.drop(columns=['index',
                   'pco2', 'ph', 'basophils', 'lactic_acid', 'bmi',
                   'creatine_kinase', 'lymphocyte', 'neutrophils'], inplace=True)
print('After drop:', df_1.shape)
print('')

# ---------- form X ----------

X_train = df_1.drop(columns=['outcome'])
print('X_train :', X_train.shape)

# ---------- form y ----------

y_train = df_1['outcome']
print('y_train :', y_train.shape)
print('')
print('y_train :', np.unique(y_train, return_counts=True))
print('y_train :', Counter(y_train))
print(y_train.value_counts(normalize=True))

Before drop : (882, 51)
After drop: (882, 42)

X_train : (882, 41)
y_train : (882,)

y_train : (array([0., 1.]), array([763, 119], dtype=int64))
y_train : Counter({0.0: 763, 1.0: 119})
0.0    0.865079
1.0    0.134921
Name: outcome, dtype: float64


<br>  

## <span style='color:blue'>Section 3: Set up 15-fold cross validation</span>  

In [3]:
kfold_cv = StratifiedKFold(n_splits=15, shuffle=True, random_state=42)

<br>  

## <span style='color:blue'>Section 4: Explore K Nearest Neighbours Classifier with neighbours</span>  

In [4]:
# ---------- make pipeline ----------

pipe_line_kn = Pipeline([('knn_impute', KNNImputer(n_neighbors=5)),
                         ('oversample_SMOTE', SMOTE(random_state=42)),
                         ('ss_scale', StandardScaler()),
                         ('kn_class', KNeighborsClassifier())])

# ---------- tuning of hyperparameters ----------

pipe_line_params_kn = {'oversample_SMOTE__sampling_strategy': [1.00],
                       'kn_class__n_neighbors': [200, 300, 400, 500, 600, 700, 800, 900, 1000, 1100, 1200],
                       'kn_class__weights': ['uniform'],
                       'kn_class__algorithm': ['brute'],
                       'kn_class__p': [2]}

# ---------- instantiate gridsearchcv for recall in accordance to aim of modelling ----------

search_kn = GridSearchCV(pipe_line_kn,
                         param_grid=pipe_line_params_kn,
                         cv=kfold_cv,
                         return_train_score=True,
                         scoring='recall')

search_kn.fit(X_train, y_train)

# ---------- quick overview of recall train score, test score and overfit ----------

print('Training :', search_kn.cv_results_['mean_train_score'])
print('Validation :', search_kn.cv_results_['mean_test_score'])
print('Overfit % :', (search_kn.cv_results_['mean_test_score']-search_kn.cv_results_['mean_train_score'])/search_kn.cv_results_['mean_train_score']*100)
print('')

# ---------- details of recall train score and test score ----------

temp_df_kn = []
for j in range(len(search_kn.cv_results_['params'])):
    for i in range(kfold_cv.n_splits):
        over_fit = (search_kn.cv_results_['split'+str(i)+'_test_score'][j]-search_kn.cv_results_['split'+str(i)+'_train_score'][j])/search_kn.cv_results_['split'+str(i)+'_train_score'][j]*100
        temp_df_kn.append([search_kn.cv_results_['params'][j],
                           search_kn.cv_results_['params'][j]['oversample_SMOTE__sampling_strategy'],
                           search_kn.cv_results_['params'][j]['kn_class__n_neighbors'],
                           search_kn.cv_results_['params'][j]['kn_class__weights'],
                           search_kn.cv_results_['params'][j]['kn_class__algorithm'],
                           search_kn.cv_results_['params'][j]['kn_class__p'],
                           search_kn.cv_results_['split'+str(i)+'_train_score'][j],
                           search_kn.cv_results_['split'+str(i)+'_test_score'][j],
                           over_fit])

# ---------- save results ----------

temp_df_kn = pd.DataFrame(temp_df_kn, columns=['parameters', 'oversample_SMOTE__sampling_strategy',
                                               'kn_class__n_neighbors', 'kn_class__weights',
                                               'kn_class__algorithm', 'kn_class__p', 'training', 'validation', 'overfit_%'])
print(temp_df_kn)
temp_df_kn.to_csv('../data/code_5c_kn_train_validate_recall.csv', na_rep='NaN', index_label='index')

Training : [0.94837516 0.94477692 0.94477156 0.94417632 0.94957636 0.94718468
 0.94297512 0.93336551 0.9285607  0.91295045 0.89614436]
Validation : [0.91666667 0.925      0.91666667 0.91666667 0.93333333 0.93333333
 0.93333333 0.91666667 0.91666667 0.90833333 0.88333333]
Overfit % : [-3.34345473 -2.09328989 -2.97478162 -2.9136139  -1.7105553  -1.46237071
 -1.02248558 -1.78909993 -1.28091108 -0.50573578 -1.42957161]

                                            parameters  \
0    {'kn_class__algorithm': 'brute', 'kn_class__n_...   
1    {'kn_class__algorithm': 'brute', 'kn_class__n_...   
2    {'kn_class__algorithm': 'brute', 'kn_class__n_...   
3    {'kn_class__algorithm': 'brute', 'kn_class__n_...   
4    {'kn_class__algorithm': 'brute', 'kn_class__n_...   
..                                                 ...   
160  {'kn_class__algorithm': 'brute', 'kn_class__n_...   
161  {'kn_class__algorithm': 'brute', 'kn_class__n_...   
162  {'kn_class__algorithm': 'brute', 'kn_class__n_...   


### <span style='color:green'>Refer to slides for visualisation of analysis.</span>  

3 Jan 2022